Import Library and data

In [ ]:
import openpyxl
import copy
from openpyxl.utils import get_column_letter
from geopy.distance import geodesic

In [2]:
path = "../data/Buildings.xlsx"
def load_data():
    workbook=openpyxl.load_workbook(path)
    worksheet=workbook["Sheet1"]
    buildings=[]
    for cell in list(worksheet.columns)[4][1:]:
        start_POS = cell.value.find("POLYGON (") + 10
        end_POS = cell.value.find("))")#获取第三列的数据
        buildings_str=cell.value[start_POS:end_POS]
        polygon = buildings_str.split(',')
        buildings.append(polygon)
    workbook.close()
    return buildings

polygon_list=load_data()

Task1 Q1.Write a program to compute the spatial extent of dataset D. The spatial extent of D is the MBR of all polygons in D. Report the MBR of D.

In [3]:
#Task 1 Question 1
def cal_mbr(polygon_list, point_min_x, point_min_y, point_max_x, point_max_y):
    result=[]
    index=[]
    for polygon in polygon_list:
        for polygon_point in polygon:
            p_x, p_y = polygon_point.strip().split(" ")
            p_x = float(p_x)
            p_y = float(p_y)
            point_min_x = min(point_min_x, p_x)
            point_min_y = min(point_min_y, p_y)
            point_max_x = max(point_max_x, p_x)
            point_max_y = max(point_max_y, p_y)
        result.append("(" + str(point_min_x) + "," + str(point_min_y) + ")" + ", " + ("(" + str(point_max_x) + "," + str(point_max_y) + ")"))
    return point_min_x, point_min_y, point_max_x, point_max_y, result

In [4]:
max_value = 1e12
min_value = -1e12
min_x, min_y, max_x, max_y, MBR=cal_mbr(polygon_list,max_value,max_value,min_value,min_value)
print("MBR of all Polygons is " + str(min_x) + ", " + str(min_y) + " " + str(max_x) + ", " + str(max_y))

MBR of all Polygons is 113.9310037, 22.1973011 114.3763554, 22.5069962


Task1 Q2. Create dataset D’ which adds an MBR column for the polygon in each record in D. Output the spreadsheet file for the new dataset D’.

In [5]:
#Task 1 Question 2
mbr_path = "../data/Buildings_MBR.xlsx"
def copy_xslx(source,dest):
    wb = openpyxl.load_workbook(source)
    wb2 = openpyxl.Workbook()
    sheetnames = wb.sheetnames
    for sheetname in sheetnames:
        print(sheetname)
        sheet = wb[sheetname]
        sheet2 = wb2.create_sheet(sheetname)
        sheet2.sheet_properties.tabColor = sheet.sheet_properties.tabColor
        wm = list(sheet.merged_cells)
        if len(wm) > 0:
            for i in range(0, len(wm)):
                cell2 = str(wm[i]).replace('(<CellRange ', '').replace('>,)', '')
                sheet2.merge_cells(cell2)
        for i, row in enumerate(sheet.iter_rows()):
            sheet2.row_dimensions[i+1].height = sheet.row_dimensions[i+1].height
            for j, cell in enumerate(row):
                sheet2.column_dimensions[get_column_letter(j+1)].width = sheet.column_dimensions[get_column_letter(j+1)].width+1
                sheet2.cell(row=i + 1, column=j + 1, value=cell.value)
                source_cell = sheet.cell(i+1, j+1)
                target_cell = sheet2.cell(i+1, j+1)
                target_cell.fill = copy.copy(source_cell.fill)
                # if source_cell.has_style:
                #     target_cell._style = copy.copy(source_cell._style)
                #     target_cell.font = copy.copy(source_cell.font)
                #     target_cell.border = copy.copy(source_cell.border)
                #     target_cell.fill = copy.copy(source_cell.fill)
                #     target_cell.number_format = copy.copy(source_cell.number_format)
                #     target_cell.protection = copy.copy(source_cell.protection)
                #     target_cell.alignment = copy.copy(source_cell.alignment)
    if 'Sheet' in wb2.sheetnames:
        del wb2['Sheet']
    wb2.save(dest)
    wb.close()
    wb2.close()
    print('Copy done.')
def add_col(data,dest):
    data_sheet = openpyxl.load_workbook(dest)
    booksheet = data_sheet.worksheets[0]
    for i, row in enumerate(booksheet.iter_rows()):
        if i==0:
            booksheet.cell(row=i + 1, column=6, value="MBR")
        else:
            booksheet.cell(row=i + 1, column=6, value=data[i-1])
    data_sheet.save(dest)
    data_sheet.close()
    print('Insert done.')
copy_xslx(path,mbr_path)
add_col(MBR,mbr_path)

Sheet1
Copy done.
Insert done.


Task1 Q3.  Let n be the resolution for recursive decomposition of the space as defined by the spatial extent of D. What are the sizes (in cm by cm) of the smallest Peano cells for n= 16, 23 and 28 respectively? Show your calculation steps. Please also discuss which resolution value is suitable for D.

In [23]:
# Task 1 Question 3
def cal_dist( min_x, min_y, max_x, max_y):
    x_dist= geodesic((min_y,min_x), (min_y,max_x)).m*100 #convert to cm
    y_dist = geodesic((min_y,min_x), (max_y,min_x)).m*100 #convert to cm
    return x_dist,y_dist
def cal_peano(n, x_dist,y_dist):
    smallest_peano=(x_dist / 2**n) * (y_dist / 2**n)
    return smallest_peano

In [25]:
x_dist,y_dist=cal_dist( min_x, min_y, max_x, max_y)
print("x: ",x_dist," y: ",y_dist)
cell_size_16 = cal_peano(16, x_dist,y_dist)
print("When n = 16, the cell_size is " + str(cell_size_16))
cell_size_23 = cal_peano(23, x_dist,y_dist)
print("When n = 23, the cell_size is " + str(cell_size_23))
cell_size_28 = cal_peano(28, x_dist,y_dist)
print("When n = 28, the cell_size is " + str(cell_size_28))

x:  4592407.166791219  y:  3429410.4542472675
When n = 16, the cell_size is 3666.90781618326
When n = 23, the cell_size is 0.22381029151509155
When n = 28, the cell_size is 0.0002185647378077066


The most suitable resolution among the above options is 23. When n=16, the precision is not enough, it may be more than 1000 cm. For n=23, the precision is enough to use, only no more than 1 cm. For n=28, the resolution is too high, it needs a lot of computing resources, which is a waste.

Task 2 Write a program to generate the base-5 z-value for each polygon, for n = 16, 23 and 28 respectively. We use only one z-value for each object based on its MBR. Add three columns of z-values to D’ for these three different resolution levels. Output the spreadsheet file with the new columns.

In [ ]:
# Task 2
